<a href="https://colab.research.google.com/github/truunganh/sentiment-analysis-project/blob/main/crawling%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install selenium webdriver-manager pandas undetected-chromedriver



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import time
import pickle
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

# Khởi tạo các options cho Chrome
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Chạy Chrome ở chế độ không giao diện (optional)
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Để tránh bị phát hiện
# Khởi tạo trình duyệt với undetected_chromedriver
driver = uc.Chrome(service=ChromeDriverManager().install(), options=chrome_options)

# Mở trang đăng nhập Twitter
driver.get("https://twitter.com/login")

# Đợi trang tải
time.sleep(3)

# Nhập username và password
username = driver.find_element(By.NAME, "text")
username.send_keys("truungganh")  # Thay đổi với username của bạn
username.send_keys(Keys.RETURN)
time.sleep(2)

password = driver.find_element(By.NAME, "password")
password.send_keys("Dipperpines3")  # Thay đổi với password của bạn
password.send_keys(Keys.RETURN)

# Đợi cho đến khi đăng nhập thành công (có thể chỉnh thời gian chờ nếu cần thiết)
time.sleep(5)

# Lưu cookies vào file sau khi đăng nhập thành công
cookies = driver.get_cookies()
with open("twitter_cookies.pkl", "wb") as cookie_file:
    pickle.dump(cookies, cookie_file)

print("Cookies đã được lưu thành công!")

# Đóng trình duyệt sau khi lưu cookies
driver.quit()


Cookies đã được lưu thành công!


In [ ]:
import time
import pickle
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Khởi tạo Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Bật nếu muốn chạy ẩn
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(service=ChromeDriverManager().install(), options=chrome_options)

# Mở trang Twitter
driver.get("https://twitter.com")

# Thêm cookies nếu đã có (đăng nhập trước rồi lưu cookies)
with open("twitter_cookies.pkl", "rb") as cookie_file:
    cookies = pickle.load(cookie_file)
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://twitter.com")
time.sleep(3)

hashtags = ["flood", "earthquake", "typhoon", "wildfire", "tornado", "landslide", "disaster"]
tweets = []

for hashtag in hashtags:
    search_url = f"https://twitter.com/search?q=%23{hashtag}%20lang%3Aen&src=typed_query&f=live"
    #search_url = f"https://twitter.com/search?q=%23{hashtag}&src=typed_query&f=live"
    driver.get(search_url)
    time.sleep(3)

    scroll_count = 20
    last_height = driver.execute_script("return document.body.scrollHeight")

    for _ in range(scroll_count):
        tweet_elements = driver.find_elements(By.XPATH, "//article[@role='article']")

        for tweet in tweet_elements:
            try:
                user_name = tweet.find_element(By.XPATH, ".//div[@dir='ltr']/span").text
                tweet_text = tweet.find_element(By.XPATH, ".//div[@lang]").text

                # Lấy tweet link để extract tweet_id
                link_element = tweet.find_element(By.XPATH, ".//a[@href and contains(@href,'/status/')]")
                link = link_element.get_attribute('href')

                # Tweet ID là số cuối cùng trong link
                tweet_id = link.split('/')[-1]

                hashtags_in_tweet = [h.text for h in tweet.find_elements(By.XPATH, ".//a[@dir='ltr']")]

                time_posted = tweet.find_element(By.XPATH, ".//time").get_attribute('datetime')

                # Xác định tweet_type: Retweet, Reply, Quote hoặc Original
                if tweet.find_elements(By.XPATH, ".//div[contains(text(),'Retweeted')]"):
                    tweet_type = "Retweet"
                elif tweet.find_elements(By.XPATH, ".//div[contains(@aria-label, 'Replying to')]"):
                    tweet_type = "Reply"
                elif tweet.find_elements(By.XPATH, ".//div[contains(text(),'Quote Tweet')]"):
                    tweet_type = "Quote"
                else:
                    tweet_type = "Original"

                if len(tweet_text) >= 100:
                    tweets.append({
                        'tweet_id': tweet_id,
                        'user_name': user_name,
                        'tweet_text': tweet_text,
                        'hashtags': hashtags_in_tweet,
                        'time_posted': time_posted,
                        'link': link,
                        'tweet_type': tweet_type
                    })
            except Exception as e:
                print(f"Error: {e}")

        # Cuộn trang
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

df = pd.DataFrame(tweets)
df.to_csv("tadatacr.csv", index=False)
print(df)

driver.quit()


Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//time"}
  (Session info: chrome=136.0.7103.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0055FC83+61635]
	GetHandleVerifier [0x0055FCC4+61700]
	(No symbol) [0x003805D3]
	(No symbol) [0x003C899E]
	(No symbol) [0x003C8D3B]
	(No symbol) [0x003BE111]
	(No symbol) [0x003ED2E4]
	(No symbol) [0x003BE034]
	(No symbol) [0x003ED514]
	(No symbol) [0x0040E61B]
	(No symbol) [0x003ED096]
	(No symbol) [0x003BC840]
	(No symbol) [0x003BD6A4]
	GetHandleVerifier [0x007E45A3+2701795]
	GetHandleVerifier [0x007DFD26+2683238]
	GetHandleVerifier [0x007FAA6E+2793134]
	GetHandleVerifier [0x00576945+155013]
	GetHandleVerifier [0x0057D02D+181357]
	GetHandleVerifier [0x005674D8+92440]
	GetHandleVerifier [0x00567680+92864]
	GetHandleVerifier [0x00552070+5296]
	BaseThreadInitThunk [0x76E85D